# LangGraph 100本ノック（解答編）

このJupyter Notebookは、LangGraphの基礎から応用までを体系的に学ぶための100問の演習問題とその解答、解説を提供します。LangGraphを用いた複雑なエージェントワークフローの構築スキルを習得しましょう。

## 目次

*   **第1章: グラフの基本要素 (問題 001-020):** `StateGraph`, `State`, ノード, エッジの定義。最もシンプルなグラフの構築と実行。
*   **第2章: グラフの制御フロー (問題 021-040):** 条件付きエッジによる分岐、自己修正ループ、エラーハンドリング、人間による介入(`Interrupt`)。
*   **第3章: ツールを使うシングルエージェント (問題 041-060):** `ToolNode`の活用、ReAct型エージェント、計画と実行(Plan-and-Execute)型エージェントの構築。
*   **第4章: マルチエージェント・ワークフロー (問題 061-090):** スーパーバイザー型によるタスク割り振り、複数エージェントによる対話シミュレーション、階層型エージェント。
*   **第5章: 発展的なグラフ技術 (問題 091-100):** Agent Swarm、永続化(`Checkpointer`)による状態保存と再開、ストリーミング、カスタムエージェント。

In [ ]:
# 必要なライブラリのインストール
%pip install -U langchain langgraph openai python-dotenv

# APIキーの設定
# .envファイルから環境変数を読み込む
from dotenv import load_dotenv
import os

load_dotenv()

# 例: OpenAI APIキー
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") # 必要に応じてコメント解除
# または直接設定する場合
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

print("ライブラリのインストールとAPIキー設定の準備が完了しました。")
print("必要に応じて、上記のAPIキー設定部分を編集してください。")

### ■ 問題001: 最小構成のLangGraphグラフの構築

LangGraphの最も基本的な構成要素である`StateGraph`と`State`を理解し、シンプルなグラフを構築してみましょう。この問題では、入力された文字列をそのまま出力するだけの、単一のノードを持つグラフを作成します。

In [ ]:
# 解答欄001

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    print(f"simple_node: {state["messages"][-1].content}")
    return {"messages": [state["messages"][-1]]}

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.____("simple_node", simple_node)

# エントリポイントの設定
workflow.____("simple_node")

# 終了ポイントの設定
workflow.____("simple_node", ____)

# グラフのコンパイル
app = workflow.____()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [("user", "Hello, LangGraph!")]}
for s in app.____(inputs):
    print(s)

# 最終結果の確認
final_state = app.____(inputs)
print(f"Final State: {final_state}")


<details><summary>解答001</summary>

``````python
# 解答001

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    # グラフの状態を保持する辞書
    # ここでは、入力メッセージを保持する
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def simple_node(state: GraphState):
    # 入力されたメッセージをそのまま返すノード
    print(f"simple_node: {state["messages"][-1].content}")
    return {"messages": [state["messages"][-1]]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("simple_node", simple_node)

# エントリポイントの設定
workflow.set_entry_point("simple_node")

# 終了ポイントの設定
workflow.add_edge("simple_node", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [("user", "Hello, LangGraph!")]}
for s in app.stream(inputs):
    print(s)

# 最終結果の確認
final_state = app.invoke(inputs)
print(f"Final State: {final_state}")
``````
</details>

<details><summary>解説001</summary>

#### この問題のポイント
*   **学習内容:** この問題では、`StateGraph`、`TypedDict`を用いた`State`の定義、`add_node`、`set_entry_point`、`add_edge`、`END`といったLangGraphの最も基本的なAPIを学びます。また、`Annotated`と`add_messages`を使ってメッセージ履歴を管理する方法も理解します。
*   **コード解説:**
    *   `GraphState`は、グラフ全体で共有される状態を定義します。`TypedDict`を使うことで、状態のスキーマを明確にできます。`messages: Annotated[list, add_messages]`は、LangChainのメッセージ形式のリストを状態として持ち、新しいメッセージが追加されるたびに自動的にリストの末尾に追加されるように設定しています。
    *   `simple_node`関数は、グラフのノードとして機能します。`state`引数として現在のグラフの状態を受け取り、新しい状態を辞書として返します。ここでは、入力された最後のメッセージをそのまま返しています。
    *   `StateGraph(GraphState)`でグラフのインスタンスを作成し、`GraphState`で定義した状態スキーマを渡します。
    *   `workflow.add_node("simple_node", simple_node)`で、`simple_node`関数を`simple_node`という名前のノードとしてグラフに追加します。
    *   `workflow.set_entry_point("simple_node")`は、グラフの実行が開始される最初のノードを指定します。
    *   `workflow.add_edge("simple_node", END)`は、`simple_node`の実行が完了したらグラフを終了することを示します。`END`はLangGraphが提供する特別な終了ノードです。
    *   `app = workflow.compile()`で、定義したワークフローを実行可能なアプリケーションにコンパイルします。
    *   `app.stream(inputs)`は、グラフの実行過程をストリーミングで受け取ることができます。`app.invoke(inputs)`は、グラフの実行が完了した最終状態を返します。
---
</details>

### ■ 問題002: 複数のノードを持つシーケンシャルグラフの構築

前の問題で学んだ基本的なグラフ構築に加えて、複数のノードを直列に接続し、データがノード間をどのように流れるかを理解しましょう。ここでは、入力された文字列を加工する2つのノード（例：大文字化、逆順化）を持つグラフを作成します。

In [ ]:
# 解答欄002

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]

# --- ノード定義 (Nodes) ---
def uppercase_node(state: GraphState):
    last_message_content = state["messages"][-1].content
    print(f"uppercase_node: {last_message_content}")
    return {"messages": [____(content=last_message_content.upper())]}

def reverse_node(state: GraphState):
    last_message_content = state["messages"][-1].content
    print(f"reverse_node: {last_message_content}")
    return {"messages": [____(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.____("uppercase", uppercase_node)
workflow.____("reverse", reverse_node)

# エントリポイントの設定
workflow.____("uppercase")

# エッジの追加 (直列接続)
workflow.____("uppercase", "reverse")
workflow.____("reverse", ____)

# グラフのコンパイル
app = workflow.____()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [____(content="Hello LangGraph")]}
for s in app.____(inputs):
    print(s)

final_state = app.____(inputs)
print(f"Final State: {final_state}")


<details><summary>解答002</summary>

``````python
# 解答002

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- ノード定義 (Nodes) ---
def uppercase_node(state: GraphState):
    # 最新のメッセージを大文字に変換するノード
    last_message_content = state["messages"][-1].content
    print(f"uppercase_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content.upper())]}

def reverse_node(state: GraphState):
    # 最新のメッセージを逆順にするノード
    last_message_content = state["messages"][-1].content
    print(f"reverse_node: {last_message_content}")
    return {"messages": [AIMessage(content=last_message_content[::-1])]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("uppercase", uppercase_node)
workflow.add_node("reverse", reverse_node)

# エントリポイントの設定
workflow.set_entry_point("uppercase")

# エッジの追加 (直列接続)
workflow.add_edge("uppercase", "reverse")
workflow.add_edge("reverse", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
inputs = {"messages": [HumanMessage(content="Hello LangGraph")]}
for s in app.stream(inputs):
    print(s)

final_state = app.invoke(inputs)
print(f"Final State: {final_state}")
``````
</details>

<details><summary>解説002</summary>

#### この問題のポイント
*   **学習内容:** 複数のノードを`add_edge`で直列に接続する方法と、ノード間で状態がどのように引き継がれるかを学びます。`HumanMessage`と`AIMessage`を使って、メッセージの送信元を明示する方法も理解します。
*   **コード解説:**
    *   `uppercase_node`と`reverse_node`は、それぞれ入力メッセージを大文字化、逆順化する処理を行います。重要なのは、各ノードが新しい`AIMessage`を作成して状態に返す点です。これにより、次のノードは前のノードの処理結果を`state["messages"][-1]`で取得できます。
    *   `workflow.add_edge("uppercase", "reverse")`は、`uppercase`ノードの実行が完了したら、次に`reverse`ノードを実行するように指示します。このようにして、処理の流れを定義します。
    *   入力メッセージを`HumanMessage`として渡すことで、ユーザーからの入力であることを明示しています。ノードからの出力は`AIMessage`として返され、メッセージ履歴にAIの応答として記録されます。
---
</details>

### ■ 問題003: 条件付きエッジによる分岐の導入

LangGraphの強力な機能の一つである条件付きエッジを導入し、グラフの実行パスを動的に制御する方法を学びましょう。ここでは、入力された数値が偶数か奇数かによって、異なる処理を行うグラフを作成します。

In [ ]:
# 解答欄003

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]
    number: int # 新たに数値を保持する状態を追加

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    # 入力メッセージから数値を抽出し、状態に保存するノード
    try:
        num = int(state["messages"][-1].content)
        print(f"check_number: Extracted number {num}")
        return {"number": num}
    except ValueError:
        print("check_number: Invalid input, not a number.")
        return {"number": 0} # エラー時は0として扱うか、適切なエラーハンドリングを実装

def even_node(state: GraphState):
    # 偶数だった場合の処理ノード
    print(f"even_node: Number {state["number"]} is even.")
    return {"messages": [____(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    # 奇数だった場合の処理ノード
    print(f"odd_node: Number {state["number"]} is odd.")
    return {"messages": [____(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    # 数値の状態に基づいて次のノードを決定する
    if state["number"] % 2 == 0:
        print("Routing to even_node")
        return "even_node"
    else:
        print("Routing to odd_node")
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.____("check_number", check_number)
workflow.____("even_node", even_node)
workflow.____("odd_node", odd_node)

# エントリポイントの設定
workflow.____("check_number")

# 条件付きエッジの追加
workflow.____(
    "check_number", # 遷移元のノード
    ____,   # ルーター関数
    {
        "even_node": "even_node", # ルーター関数の戻り値とノード名のマッピング
        "odd_node": "odd_node"
    }
 )

# 各分岐からの終了エッジ
workflow.____("even_node", ____)
workflow.____("odd_node", ____)

# グラフのコンパイル
app = workflow.____()

# --- グラフの実行と結果表示 ---
print("\n--- 偶数のテスト ---")
inputs_even = {"messages": [____(content="42")]}
for s in app.____(inputs_even):
    print(s)
final_state_even = app.____(inputs_even)
print(f"Final State (Even): {final_state_even}")

print("\n--- 奇数のテスト ---")
inputs_odd = {"messages": [____(content="77")]}
for s in app.____(inputs_odd):
    print(s)
final_state_odd = app.____(inputs_odd)
print(f"Final State (Odd): {final_state_odd}")


<details><summary>解答003</summary>

``````python
# 解答003

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    number: int # 新たに数値を保持する状態を追加

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    # 入力メッセージから数値を抽出し、状態に保存するノード
    try:
        num = int(state["messages"][-1].content)
        print(f"check_number: Extracted number {num}")
        return {"number": num}
    except ValueError:
        print("check_number: Invalid input, not a number.")
        return {"number": 0} # エラー時は0として扱うか、適切なエラーハンドリングを実装

def even_node(state: GraphState):
    # 偶数だった場合の処理ノード
    print(f"even_node: Number {state["number"]} is even.")
    return {"messages": [AIMessage(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    # 奇数だった場合の処理ノード
    print(f"odd_node: Number {state["number"]} is odd.")
    return {"messages": [AIMessage(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    # 数値の状態に基づいて次のノードを決定する
    if state["number"] % 2 == 0:
        print("Routing to even_node")
        return "even_node"
    else:
        print("Routing to odd_node")
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("check_number", check_number)
workflow.add_node("even_node", even_node)
workflow.add_node("odd_node", odd_node)

# エントリポイントの設定
workflow.set_entry_point("check_number")

# 条件付きエッジの追加
workflow.add_conditional_edges(
    "check_number", # 遷移元のノード
    route_number,   # ルーター関数
    {
        "even_node": "even_node", # ルーター関数の戻り値とノード名のマッピング
        "odd_node": "odd_node"
    }
)

# 各分岐からの終了エッジ
workflow.add_edge("even_node", END)
workflow.add_edge("odd_node", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- 偶数のテスト ---")
inputs_even = {"messages": [HumanMessage(content="42")]}
for s in app.stream(inputs_even):
    print(s)
final_state_even = app.invoke(inputs_even)
print(f"Final State (Even): {final_state_even}")

print("\n--- 奇数のテスト ---")
inputs_odd = {"messages": [HumanMessage(content="77")]}
for s in app.stream(inputs_odd):
    print(s)
final_state_odd = app.invoke(inputs_odd)
print(f"Final State (Odd): {final_state_odd}")
``````
</details>

<details><summary>解説003</summary>

#### この問題のポイント
*   **学習内容:** `add_conditional_edges`を使用して、グラフの実行パスを動的に制御する方法を学びます。ルーター関数がどのように次のノードを決定するのか、そして状態が分岐間でどのように共有されるかを理解します。
*   **コード解説:**
    *   `GraphState`に`number`という新しいキーを追加し、入力された数値を保持するようにしました。
    *   `check_number`ノードは、入力メッセージから数値を抽出し、`number`状態を更新します。
    *   `even_node`と`odd_node`は、それぞれ偶数と奇数だった場合の最終処理を行います。
    *   `route_number`関数がルーターとして機能します。この関数は現在の`state`を受け取り、次に実行すべきノードの名前（文字列）を返します。LangGraphは、この戻り値に基づいて適切なエッジを辿ります。
    *   `workflow.add_conditional_edges("check_number", route_number, {"even_node": "even_node", "odd_node": "odd_node"})`は、`check_number`ノードの後に`route_number`関数を実行し、その戻り値が`"even_node"`なら`even_node`へ、`"odd_node"`なら`odd_node`へ遷移するように設定しています。
---
</details>

### ■ 問題004: グラフ内でのLLMの利用（シンプルなチャットボット）

LangGraphのノード内で大規模言語モデル（LLM）を呼び出す方法を学び、シンプルなチャットボットを構築しましょう。ここでは、ユーザーからの入力に対してLLMが応答を生成し、その応答を返すグラフを作成します。

In [ ]:
# 解答欄004
from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
import os

# 環境変数からAPIキーを読み込む（.envファイルが設定されている前提）
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") # 必要に応じてコメント解除

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]

# --- LLMの初期化 ---
llm = ____(model="gpt-3.5-turbo")

# --- ノード定義 (Nodes) ---
def llm_node(state: GraphState):
    # LLMを呼び出し、応答を生成するノード
    print(f"llm_node: Calling LLM with messages: {state["messages"]}")
    response = llm.____(state["messages"])
    print(f"llm_node: LLM response: {response.content}")
    return {"messages": [____]}

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

# ノードの追加
workflow.____("llm_responder", llm_node)

# エントリポイントの設定
workflow.____("llm_responder")

# 終了ポイントの設定
workflow.____("llm_responder", ____)

# グラフのコンパイル
app = workflow.____()

# --- グラフの実行と結果表示 ---
print("\n--- チャットボットのテスト ---")
inputs = {"messages": [____(content="こんにちは、あなたの名前は何ですか？")]}
for s in app.____(inputs):
    print(s)

final_state = app.____(inputs)
print(f"Final State: {final_state}")

print("\n--- 別の質問 ---")
inputs2 = {"messages": [____(content="今日の天気は？")]}
for s in app.____(inputs2):
    print(s)

final_state2 = app.____(inputs2)
print(f"Final State: {final_state2}")


<details><summary>解答004</summary>

``````python
# 解答004

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
import os

# 環境変数からAPIキーを読み込む（.envファイルが設定されている前提）
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") # 必要に応じてコメント解除

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]

# --- LLMの初期化 ---
llm = ChatOpenAI(model="gpt-3.5-turbo")

# --- ノード定義 (Nodes) ---
def llm_node(state: GraphState):
    # LLMを呼び出し、応答を生成するノード
    print(f"llm_node: Calling LLM with messages: {state["messages"]}")
    response = llm.invoke(state["messages"])
    print(f"llm_node: LLM response: {response.content}")
    return {"messages": [response]}

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

# ノードの追加
workflow.add_node("llm_responder", llm_node)

# エントリポイントの設定
workflow.set_entry_point("llm_responder")

# 終了ポイントの設定
workflow.add_edge("llm_responder", END)

# グラフのコンパイル
app = workflow.compile()

# --- グラフの実行と結果表示 ---
print("\n--- チャットボットのテスト ---")
inputs = {"messages": [HumanMessage(content="こんにちは、あなたの名前は何ですか？")]}
for s in app.stream(inputs):
    print(s)

final_state = app.invoke(inputs)
print(f"Final State: {final_state}")

print("\n--- 別の質問 ---")
inputs2 = {"messages": [HumanMessage(content="今日の天気は？")]}
for s in app.stream(inputs2):
    print(s)

final_state2 = app.invoke(inputs2)
print(f"Final State: {final_state2}")
``````
</details>

<details><summary>解説004</summary>

#### この問題のポイント
*   **学習内容:** LangGraphのノード内でLangChainのLLMを統合し、チャットボットのような対話システムを構築する方法を学びます。`ChatOpenAI`の基本的な使い方と、メッセージ履歴をLLMに渡す方法を理解します。
*   **コード解説:**
    *   `ChatOpenAI`をインポートし、使用するLLMモデル（例: `gpt-3.5-turbo`）を指定して初期化します。APIキーは環境変数から読み込むか、直接設定します。
    *   `llm_node`関数内で、`llm.invoke(state["messages"])`を呼び出すことで、現在のメッセージ履歴全体をLLMに渡しています。これにより、LLMは過去の会話履歴を考慮した応答を生成できます。
    *   LLMからの応答は`AIMessage`オブジェクトとして返されるため、そのまま`messages`状態に追加することで、次のターンでLLMがその応答を参照できるようになります。
    *   このシンプルなグラフは、ユーザー入力 -> LLM応答 -> 終了という一連の流れを表現しており、LangGraphでLLMベースのアプリケーションを構築する際の基礎となります。
---
</details>

### ■ 問題005: グラフの可視化とデバッグ

構築したLangGraphグラフの構造を視覚的に確認し、デバッグに役立てる方法を学びましょう。ここでは、これまでに作成したグラフのいずれか（例：問題003の条件分岐グラフ）を可視化し、その構造を理解します。

In [ ]:
# 解答欄005

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, ____]
    number: int

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    try:
        num = int(state["messages"][-1].content)
        return {"number": num}
    except ValueError:
        return {"number": 0}

def even_node(state: GraphState):
    return {"messages": [____(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    return {"messages": [____(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    if state["number"] % 2 == 0:
        return "even_node"
    else:
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = ____(GraphState)

workflow.____("check_number", check_number)
workflow.____("even_node", even_node)
workflow.____("odd_node", odd_node)

workflow.____("check_number")

workflow.____(
    "check_number",
    ____,
    {
        "even_node": "even_node",
        "odd_node": "odd_node"
    }
)

workflow.____("even_node", ____)
workflow.____("odd_node", ____)

app = workflow.____()

# --- グラフの可視化 ---
# グラフを画像として表示
# graphvizがインストールされている必要があります: pip install pygraphviz pydotplus graphviz
# また、システムにGraphvizがインストールされている必要があります。
# Windows: https://graphviz.org/download/
# Mac: brew install graphviz
# Linux: sudo apt-get install graphviz
try:
    ____(____(app.____().____()))
    print("グラフが正常に可視化されました。")
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

# --- グラフの実行と結果表示 (オプション) ---
# 可視化したグラフが正しく動作するか確認するために、再度実行してみる
print("\n--- 偶数のテスト (可視化後の確認) ---")
inputs_even = {"messages": [____(content="10")]}
for s in app.____(inputs_even):
    print(s)
final_state_even = app.____(inputs_even)
print(f"Final State: {final_state_even}")


<details><summary>解答005</summary>

``````python
# 解答005

from typing import TypedDict, Annotated
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, HumanMessage, AIMessage
from IPython.display import Image, display

# --- 状態定義 (State) ---
class GraphState(TypedDict):
    messages: Annotated[list, add_messages]
    number: int

# --- ノード定義 (Nodes) ---
def check_number(state: GraphState):
    try:
        num = int(state["messages"][-1].content)
        return {"number": num}
    except ValueError:
        return {"number": 0}

def even_node(state: GraphState):
    return {"messages": [AIMessage(content=f"The number {state["number"]} is even.")]}

def odd_node(state: GraphState):
    return {"messages": [AIMessage(content=f"The number {state["number"]} is odd.")]}

# --- 条件付きエッジのルーター関数 ---
def route_number(state: GraphState):
    if state["number"] % 2 == 0:
        return "even_node"
    else:
        return "odd_node"

# --- グラフ構築 (Graph) ---
workflow = StateGraph(GraphState)

workflow.add_node("check_number", check_number)
workflow.add_node("even_node", even_node)
workflow.add_node("odd_node", odd_node)

workflow.set_entry_point("check_number")

workflow.add_conditional_edges(
    "check_number",
    route_number,
    {
        "even_node": "even_node",
        "odd_node": "odd_node"
    }
)

workflow.add_edge("even_node", END)
workflow.add_edge("odd_node", END)

app = workflow.compile()

# --- グラフの可視化 ---
# グラフを画像として表示
# graphvizがインストールされている必要があります: pip install pygraphviz pydotplus graphviz
# また、システムにGraphvizがインストールされている必要があります。
# Windows: https://graphviz.org/download/
# Mac: brew install graphviz
# Linux: sudo apt-get install graphviz
try:
    display(Image(app.get_graph().draw_png()))
    print("グラフが正常に可視化されました。")
except Exception as e:
    print(f"グラフの可視化に失敗しました。Graphvizが正しくインストールされているか確認してください。エラー: {e}")

# --- グラフの実行と結果表示 (オプション) ---
# 可視化したグラフが正しく動作するか確認するために、再度実行してみる
print("\n--- 偶数のテスト (可視化後の確認) ---")
inputs_even = {"messages": [HumanMessage(content="10")]}
for s in app.stream(inputs_even):
    print(s)
final_state_even = app.invoke(inputs_even)
print(f"Final State: {final_state_even}")
``````
</details>

<details><summary>解説005</summary>

#### この問題のポイント
*   **学習内容:** `app.get_graph().draw_png()`を使用してLangGraphのグラフ構造を画像として可視化する方法を学びます。これにより、複雑なグラフのデバッグや理解が容易になります。
*   **コード解説:**
    *   この問題では、問題003で作成した条件分岐グラフを再利用しています。これは、可視化の有用性を示すのに適した例だからです。
    *   `app.get_graph()`は、コンパイルされたグラフの内部表現を取得します。
    *   `.draw_png()`メソッドは、そのグラフ構造をPNG画像としてバイト列で返します。この機能を利用するには、システムにGraphvizがインストールされている必要があります。また、Pythonの`pygraphviz`や`pydotplus`といったライブラリも必要になる場合があります。
    *   `IPython.display.Image`と`display`を使うことで、Jupyter Notebook内で直接画像をレンダリングして表示できます。
    *   `try-except`ブロックでGraphvizのインストール状況によるエラーをハンドリングし、ユーザーに適切なメッセージを表示するようにしています。グラフが複雑になるにつれて、この可視化機能はデバッグや設計の確認に不可欠となります。
---
</details>